In [1]:
import asyncio
import json
import os
import pathlib

In [2]:
cwd = pathlib.Path().cwd()
proj_dir = cwd.parent.parent.parent
sample_objects_dir = proj_dir / 'sample_objects'
os.chdir(proj_dir)

from utils.arcgis_helpers import checkout_token
from utils.arcgis_helpers import AsyncArcGISRequester

In [3]:
nifc_token = checkout_token('NIFC_AGO', 120, 'NIFC_TOKEN', 15)

# save current state of service, so features can be replaced to replicate state.

In [10]:
async with AsyncArcGISRequester() as requester:
    snapshot_0 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/0',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )
    snapshot_1 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/1',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )
    snapshot_2 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/2',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )
    snapshot_3 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/3',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )

Entering async context...


C:\Program Files\Python312\Lib\json\decoder.py:354: RuntimeWarning: coroutine 'AsyncArcGISRequester.applyEdits_request' was never awaited
  obj, end = self.scan_once(s, idx)


Exiting async context...


In [5]:
print(sample_objects_dir)

c:\REPOS\con-j-e\ak-wildfire-values-at-risk\sample_objects


In [11]:
for name,response in {
    'perims_locs': snapshot_0,
    'buf_1': snapshot_1,
    'buf_3': snapshot_2,
    'buf_5': snapshot_3
}.items():
    feats = response['features']
    print(len(feats))
    with open(sample_objects_dir / f'{name}.json', 'w') as file:
        json.dump(feats, file)
    


29
29
29
29


# read service state snapshots and add features to service to replicate state

In [4]:
feats_dict = {
    'perims_locs': None,
    'buf_1': None,
    'buf_3': None,
    'buf_5': None
}

for name,_ in feats_dict.items():
    with open(sample_objects_dir / f'{name}.json', 'r') as file:
        feats = json.load(file)
    feats_dict[name] = feats

lyr_idx_dict = {
    'perims_locs': 0,
    'buf_1': 1,
    'buf_3': 2,
    'buf_5': 3,
}

async with AsyncArcGISRequester() as requester:
    for name, idx in lyr_idx_dict.items():
        response = await requester.applyEdits_request(
            url=fr'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/{idx}',
            token=nifc_token,
            features_to_add=feats_dict[name],
            get_oids_to_delete_query='1=1'
        )
        print(response)


Entering async context...
{'addResults': [{'objectId': 531, 'uniqueId': 531, 'globalId': '739BC3B0-7CC4-4277-836A-3D9662DD5185', 'success': True}, {'objectId': 532, 'uniqueId': 532, 'globalId': 'EAE5E48F-A87B-47AD-BAD6-9447B135F190', 'success': True}, {'objectId': 533, 'uniqueId': 533, 'globalId': '3082A0B4-1995-4B13-A7F4-201D29B33CAE', 'success': True}, {'objectId': 534, 'uniqueId': 534, 'globalId': 'C6F74980-D880-4AB8-8113-D6A7C74956A4', 'success': True}, {'objectId': 535, 'uniqueId': 535, 'globalId': 'A1428386-E310-4E49-908F-D7371090C5D2', 'success': True}, {'objectId': 536, 'uniqueId': 536, 'globalId': 'C1D8F7A4-4C99-476C-8710-B15EA6484777', 'success': True}, {'objectId': 537, 'uniqueId': 537, 'globalId': '3AF09545-134C-4250-A351-287D3ACF9112', 'success': True}, {'objectId': 538, 'uniqueId': 538, 'globalId': 'D57209CE-2803-4F8B-AE3D-A12F4D8F010D', 'success': True}, {'objectId': 539, 'uniqueId': 539, 'globalId': 'E19BD870-9D64-4E0F-B502-A1181B9B6314', 'success': True}, {'objectId': 